<a href="https://colab.research.google.com/github/MaxCojocari/covid-19_forecaster/blob/main/expl_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#https://www.kaggle.com/headsortails/covid19-tracking-germany
!mkdir -p ~/.kaggle
!cp drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download --unzip -d headsortails/covid19-tracking-germany

 52% 10.0M/19.3M [00:00<00:00, 26.8MB/s]
100% 19.3M/19.3M [00:00<00:00, 50.9MB/s]


In [3]:
#Import all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
#Upload the data frame
covid = pd.read_csv('covid_de.csv')
covid

,state,county,age_group,gender,date,cases,deaths,recovered
0,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-03-27,1,0,1
1,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-03-28,1,0,1
2,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-04-03,1,0,1
3,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-10-18,1,0,1
4,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,F,2020-10-22,1,0,1
...,...,...,...,...,...,...,...,...
1830839,Thueringen,SK Weimar,NaN,F,2022-04-01,1,0,0
1830840,Thueringen,SK Weimar,NaN,M,2022-02-02,1,0,1
1830841,Thueringen,SK Weimar,NaN,M,2022-03-11,1,0,0
1830842,Thueringen,SK Weimar,NaN,M,2022-03-28,2,0,0


In [5]:
#Display overall info about your dataframe
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830844 entries, 0 to 1830843
Data columns (total 8 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   state      object
 1   county     object
 2   age_group  object
 3   gender     object
 4   date       object
 5   cases      int64 
 6   deaths     int64 
 7   recovered  int64 
dtypes: int64(3), object(5)
memory usage: 111.7+ MB


Observations on the covid dataset:

The dataset consists of 1.830.844 logs, and 8 columns;
 

*   Numerical columns : 'cases', 'deaths', and 'recovered';
*   3 Numerical columns : 'cases', 'deaths', and 'recovered';
*   1 Timestamp column : 'date';
*   4 Categorical columns : 'state', 'county', 'gender' and 'age_group'.

The dataset informations allow us to see the data types, which we can already state that they are not correctly assigned. In the next steps, we will address the data types in more detail.

In [6]:
#Find out how many None raws you have in the data
covid.isna().sum()

state            0
county           0
age_group    11488
gender       78329
date             0
cases            0
deaths           0
recovered        0
dtype: int64

In [7]:
# Sample of rows with missing values from covid
display(covid[covid['gender'].isnull()].head(6))
display(covid[covid['age_group'].isnull()].head(6))

,state,county,age_group,gender,date,cases,deaths,recovered
474,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,NaN,2022-01-23,1,0,1
475,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,NaN,2022-01-26,1,0,1
476,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,NaN,2022-02-09,1,0,1
477,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,NaN,2022-02-15,1,0,1
478,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,NaN,2022-02-16,3,0,3
479,Baden-Wuerttemberg,LK Alb-Donau-Kreis,00-04,NaN,2022-02-18,1,0,1


,state,county,age_group,gender,date,cases,deaths,recovered
5003,Baden-Wuerttemberg,LK Alb-Donau-Kreis,NaN,F,2020-10-26,1,0,1
5004,Baden-Wuerttemberg,LK Alb-Donau-Kreis,NaN,F,2020-12-24,1,0,1
5005,Baden-Wuerttemberg,LK Alb-Donau-Kreis,NaN,F,2022-01-24,1,0,1
5006,Baden-Wuerttemberg,LK Alb-Donau-Kreis,NaN,F,2022-02-17,1,0,1
5007,Baden-Wuerttemberg,LK Alb-Donau-Kreis,NaN,F,2022-02-25,1,0,1
5008,Baden-Wuerttemberg,LK Alb-Donau-Kreis,NaN,F,2022-03-16,1,0,0


In [8]:
print('{:.2%} of rows in the covid dataset are missing data.'.format(covid.isnull().sum().sum()/len(covid)))

4.91% of rows in the covid dataset are missing data.


Observations :

We notice above that the number of rows missing data in the age_group column is very small : 11488 rows, which makes for 0.62% of the dataset rows. This means that we can simply drop these rows, as their effect on the study results will be minimal.

On the other hand, the number is bigger for the gender column, which is missing 78.329 rows (4.27%). Before we decide how to treat these rows, let's take a look at the rows with missing values. We will display, as an example, the data for the state of 'Baden-Wuerttemberg', the county of 'LK Alb-Donau-Kreis', for the age groups '05-14' and '15-34', on the following dates : '2020-10-30' and '2022-01-31'.

In [9]:
# Creating data slice
Slice = covid.query('state == "Baden-Wuerttemberg" and county == "LK Alb-Donau-Kreis" and age_group == ["05-14","15-34"] and date == ["2020-10-30", "2022-01-31"]')
Slice.sort_values(by = 'date')

,state,county,age_group,gender,date,cases,deaths,recovered
520,Baden-Wuerttemberg,LK Alb-Donau-Kreis,05-14,F,2020-10-30,2,0,2
895,Baden-Wuerttemberg,LK Alb-Donau-Kreis,05-14,M,2020-10-30,1,0,1
1242,Baden-Wuerttemberg,LK Alb-Donau-Kreis,05-14,NaN,2020-10-30,1,0,1
1390,Baden-Wuerttemberg,LK Alb-Donau-Kreis,15-34,F,2020-10-30,8,0,8
1937,Baden-Wuerttemberg,LK Alb-Donau-Kreis,15-34,M,2020-10-30,9,0,9
2391,Baden-Wuerttemberg,LK Alb-Donau-Kreis,15-34,NaN,2020-10-30,2,0,2
801,Baden-Wuerttemberg,LK Alb-Donau-Kreis,05-14,F,2022-01-31,35,0,35
1181,Baden-Wuerttemberg,LK Alb-Donau-Kreis,05-14,M,2022-01-31,43,0,43
1780,Baden-Wuerttemberg,LK Alb-Donau-Kreis,15-34,F,2022-01-31,72,0,72
2330,Baden-Wuerttemberg,LK Alb-Donau-Kreis,15-34,M,2022-01-31,54,0,54


This allows us to see that for all these instances, both genders are already present, and that the row with the missing value is either due to human error, where the doctors testing the patients for example did not note the gender of the patient, and in older gender slices, it can be due to the personal choice of the tested person to not reveal their gender, or to not identify as either of the binary gender values.

Therefore, we will assign the value 'U' to this category to indicate that their gender is unkown.

In [10]:
# Filling missing values in the gender column with the value 'U'
covid['gender'] = covid['gender'].fillna('U')

# Dropping the remaining missing values which are from the age group column
covid = covid.dropna()

# Infos on the dataset
covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1819356 entries, 0 to 1830825
Data columns (total 8 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   state      object
 1   county     object
 2   age_group  object
 3   gender     object
 4   date       object
 5   cases      int64 
 6   deaths     int64 
 7   recovered  int64 
dtypes: int64(3), object(5)
memory usage: 124.9+ MB


In [11]:
covid.isna().sum()

state        0
county       0
age_group    0
gender       0
date         0
cases        0
deaths       0
recovered    0
dtype: int64

In [12]:
# Checking for duplicates
print('number of duplicated rows in "covid" :', covid.duplicated().sum())

number of duplicated rows in "covid" : 0


As observed previously, we noticed that there are columns with wrong data types. In this step, we will convert the date columns into the right data type.

In [13]:
# Converting the 'date' columns
covid['date'] = covid['date'].astype('datetime64[D]')
covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1819356 entries, 0 to 1830825
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   state      object        
 1   county     object        
 2   age_group  object        
 3   gender     object        
 4   date       datetime64[ns]
 5   cases      int64         
 6   deaths     int64         
 7   recovered  int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 124.9+ MB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Conclusion: 
So far, we have handled the missing data, where we removed 6749 rows of data, which was about 0.43% of the entirety of the covid dataset. No duplicates have been found.

We have also converted the dates to the proper format, which makes our datatsets now ready for the next step.

In [14]:
demographics = pd.read_csv('demographics_de.csv')
demographics

,state,gender,age_group,population
0,Baden-Wuerttemberg,female,00-04,261674
1,Baden-Wuerttemberg,female,05-14,490822
2,Baden-Wuerttemberg,female,15-34,1293488
3,Baden-Wuerttemberg,female,35-59,1919649
4,Baden-Wuerttemberg,female,60-79,1182736
...,...,...,...,...
187,Thueringen,male,05-14,92545
188,Thueringen,male,15-34,214553
189,Thueringen,male,35-59,384822
190,Thueringen,male,60-79,264189


In [15]:
display(demographics.describe().round(2))

,population
count,192.00
mean,432391.73
std,557233.70
min,15906.00
25%,95457.00
50%,234596.00
75%,484169.00
max,3147565.00


In [16]:
print('The data in covid_de covers the period between :', covid.date.min(), 'and', covid.date.max(),'.')
print('The total number of COVID-19 cases in Germany is :', covid.cases.sum(), 'cases.')
print('The total number of COVID-19 deaths in Germany is :', covid.deaths.sum(), 'deaths.')
print('The total number of COVID-19 recoveries in Germany is :', covid.recovered.sum(), 'cases.')

The data in covid_de covers the period between : 2020-01-02 00:00:00 and 2022-04-02 00:00:00 .
The total number of COVID-19 cases in Germany is : 21601854 cases.
The total number of COVID-19 deaths in Germany is : 130018 deaths.
The total number of COVID-19 recoveries in Germany is : 16955496 cases.


In [17]:
# Calculating Number of Active Cases ("active")
covid['active'] = covid['cases'] - covid['recovered'] - covid['deaths']

# Display
display(covid.sample(11))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,state,county,age_group,gender,date,cases,deaths,recovered,active
516238,Bayern,LK Weilheim-Schongau,15-34,F,2022-01-27,43,0,43,0
544728,Bayern,SK Augsburg,60-79,F,2021-10-27,9,0,9,0
1523733,Rheinland-Pfalz,SK Mainz,15-34,F,2021-11-19,9,0,9,0
237758,Bayern,LK Ansbach,15-34,F,2021-04-30,12,0,12,0
118141,Baden-Wuerttemberg,LK Rastatt,60-79,M,2021-10-20,3,0,3,0
1756952,Thueringen,LK Eichsfeld,80-99,M,2021-11-19,4,0,4,0
1652657,Sachsen-Anhalt,LK Harz,05-14,M,2021-12-04,28,0,28,0
1223946,Nordrhein-Westfalen,LK Rhein-Kreis Neuss,15-34,M,2020-12-13,10,0,10,0
107507,Baden-Wuerttemberg,LK Ortenaukreis,60-79,M,2021-09-25,4,1,3,0
1356534,Nordrhein-Westfalen,SK Koeln,60-79,U,2022-01-28,2,0,2,0


In [18]:
# Grouping the data by date
cases = covid.groupby('date', as_index = False).agg({'cases' : 'sum'})
cases

,date,cases
0,2020-01-02,1
1,2020-01-05,1
2,2020-01-23,1
3,2020-01-28,2
4,2020-01-29,2
...,...,...
783,2022-03-29,267475
784,2022-03-30,251639
785,2022-03-31,219960
786,2022-04-01,149945


In [19]:
start_date = '2021-01-01'
select_data_cases = cases.loc[cases['date'] >= start_date]
select_data_cases

,date,cases
331,2021-01-01,10271
332,2021-01-02,10011
333,2021-01-03,8833
334,2021-01-04,14301
335,2021-01-05,27428
...,...,...
783,2022-03-29,267475
784,2022-03-30,251639
785,2022-03-31,219960
786,2022-04-01,149945


In [20]:
# Plot bar chart
fig = px.bar(select_data_cases, x = 'date', y = 'cases', height = 600,
             color = 'cases', 
             color_continuous_scale = px.colors.sequential.Viridis,
             title = f'Daily Cases of COVID-19 in Germany starting from {start_date}')
fig.show()

In [21]:
# Plotting daily deaths

# Grouping the data by date
deaths = covid.groupby('date', as_index = False).agg({'deaths' : 'sum'})
deaths.to_csv('/content/drive/MyDrive/PBL_sem2/deaths_covid_de.csv', index=False)
deaths

,date,deaths
0,2020-01-02,0
1,2020-01-05,0
2,2020-01-23,0
3,2020-01-28,0
4,2020-01-29,0
...,...,...
783,2022-03-29,42
784,2022-03-30,30
785,2022-03-31,28
786,2022-04-01,12


In [22]:
#Selecting rows for plotting
start_date = '2021-01-01'
select_data_deaths = deaths.loc[deaths['date'] >= start_date]
select_data_deaths

,date,deaths
331,2021-01-01,414
332,2021-01-02,589
333,2021-01-03,496
334,2021-01-04,775
335,2021-01-05,1042
...,...,...
783,2022-03-29,42
784,2022-03-30,30
785,2022-03-31,28
786,2022-04-01,12


In [23]:
# Plot bar chart
fig = px.bar(select_data_deaths, x = 'date', y = 'deaths',height = 600,
             color = 'deaths', 
             color_continuous_scale = px.colors.sequential.Jet,
             title = f'Daily Deaths of COVID-19 in Germany starting from {start_date}')
fig.show()

In [24]:
# Grouping the data by date
recovered = covid.groupby('date', as_index = False).agg({'recovered' : 'sum'})
recovered.to_csv('/content/drive/MyDrive/PBL_sem2/recovered_covid_de.csv', index=False)
recovered

,date,recovered
0,2020-01-02,1
1,2020-01-05,1
2,2020-01-23,1
3,2020-01-28,2
4,2020-01-29,2
...,...,...
783,2022-03-29,1653
784,2022-03-30,1851
785,2022-03-31,2208
786,2022-04-01,1510


In [25]:
#Selecting rows for plotting
start_date = '2021-01-01'
select_data_recovered = recovered.loc[cases['date'] >= start_date]
select_data_recovered

,date,recovered
331,2021-01-01,9857
332,2021-01-02,9422
333,2021-01-03,8337
334,2021-01-04,13526
335,2021-01-05,26386
...,...,...
783,2022-03-29,1653
784,2022-03-30,1851
785,2022-03-31,2208
786,2022-04-01,1510


In [26]:
# Plot bar chart
fig = px.bar(select_data_recovered, x = 'date', y = 'recovered',height = 600,
             color = 'recovered', 
             color_continuous_scale = px.colors.sequential.Inferno,
             title = f'Daily Recoveries of COVID-19 in Germany starting from {start_date}')
fig.show()

In [27]:
#Line plot for daily cases
import plotly.graph_objects as go

new_line = '\n'

# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=select_data_cases['date'], y=select_data_cases['cases'],
                    mode='lines',
                    name='daily cases'))
fig.add_trace(go.Scatter(x=select_data_deaths['date'], y=select_data_deaths['deaths']*100,
                    mode='lines',
                    name='deaths*100'))
fig.add_trace(go.Scatter(x=select_data_recovered['date'], y=select_data_recovered['recovered'],
                    mode='lines', name='recovered'))

fig.update_layout(title=f'Covid-19 situation in Germany between {select_data_cases.date.min()} and {select_data_cases.date.max()}',
                   xaxis_title='date',
                   yaxis_title='nr_people')


fig.show()


In [28]:
cases = cases.loc[cases['date'] >= '2021-03-01']

In [29]:
path = '/content/drive/MyDrive/PBL_sem2/cases_covid_de.csv'

In [30]:
cases.to_csv(path, index=False)